# Proximal Policy Optimization

## Introduction

### Policy gradient

$$J(\theta)=\mathbb{E}_{\pi_\theta} \left [ \sum_t r_t \right ] =v_{\pi_\theta}(s_0)=\sum_{s\in S}d(s)*v_{\pi_\theta}(s)$$

1step MDP : 

$$J(\theta)=\sum_{s\in S}d(s)*v_{\pi_\theta}(s)=\sum_{s\in S}d(s)\sum_{a\in A}\pi_\theta(s,a)*R_{s,a}$$

$$\begin{align*}
\triangledown_\theta J(\theta) &= \triangledown_\theta \sum_{s\in S}d(s)\sum_{a\in A}\pi_\theta(s,a)*R_{s,a} \\
&= \sum_{s\in S}d(s)\sum_{a\in A}\triangledown_\theta\pi_\theta(s,a)*R_{s,a} \\
&=\sum_{s\in S}d(s)\sum_{a\in A}\frac{\pi_\theta (s,a)}{\pi_\theta (s,a)}\triangledown_\theta\pi_\theta(s,a)*R_{s,a} \\
&=\sum_{s\in S}d(s)\sum_{a\in A}\pi_\theta (s,a)\frac{\triangledown_\theta\pi_\theta(s,a)}{\pi_\theta (s,a)}*R_{s,a} \\
&=\sum_{s\in S}d(s)\sum_{a\in A}\pi_\theta (s,a)\triangledown_\theta log\pi_\theta(s,a)*R_{s,a} \\
&=\mathbb{E}_{\pi_\theta}[\triangledown_\theta log\pi_\theta(s,a)*R_{s,a}]
\end{align*}$$

MDP : $$\triangledown_\theta J(\theta) = \mathbb{E}_{\pi_\theta}[\triangledown_\theta log\pi_\theta(s,a)*Q_{\pi_\theta}(s,a)]$$

### REINFOCE  Algorithm

$G_t$의 샘플을 여러 개 얻어서 평균을 내면 그 값이 실제 $Q_{\pi_\theta}(s,a)$에 근사하기 때문

$$\triangledown_\theta J(\theta) = \mathbb{E}_{\pi_\theta}[\triangledown_\theta log\pi_\theta(s,a)*G_t]$$

### Advantage Policy gradient

$$\triangledown_\theta J(\theta) = \mathbb{E}_{\pi_\theta}[\triangledown_\theta log\pi_\theta(s,a)*\{Q_{\pi_\theta}(s,a)-V_{\pi_\theta}(s)\}]$$
$$\triangledown_\theta J(\theta) = \mathbb{E}_{\pi_\theta}[\triangledown_\theta log\pi_\theta(s,a)*A_{\pi_\theta}(s,a)]$$

### Advantage Policy gradient loss

$$L^{PG}(\theta)=\mathbb{E}_{Q_{\pi_\theta}}[log\pi_\theta(s,a)*A_{\pi_\theta}(s,a)]$$

### Importance Sampling

$$\triangledown_\theta log\pi_\theta(s,a)|_{\theta_\text{old}}=\frac{\triangledown_\theta\pi_\theta(s,a)|_{\theta_\text{old}}}{\pi_{\theta_\text{old}}(s,a)}=\triangledown_{\theta}\begin{pmatrix}\frac{\pi_\theta(s,a)}{\pi_{\theta_\text{old}}(s,a)}\end{pmatrix}|_{\theta_\text{old}}$$


$$L^{IS}_{\theta_{\text{old}}}(\theta)=\mathbb{E}_{Q_{\pi_{\theta_\text{old}}}}\begin{bmatrix}\frac{\pi_\theta(s,a)}{\pi_{\theta_\text{old}}(s,a)}*A_{\pi_\theta}(s,a)\end{bmatrix}$$

### Cliping

$\theta$와 $\theta_{\text{old}}$는 비슷해야한다.

$$L^{CLIP}(\theta) = \mathbb{E}_t[\text{min}(r_t(\theta)A_t, \text{clip}(r_t(\theta),1-\epsilon,1+\epsilon)A_t)]$$

<img src='../img/PPO_1.png' width='800'> 

### Adaptive KL Penalty Coefficient

<img src='../img/ppo_2.png' width='800'> 

### Generalized Advantage Estimation

n-step TD 대신 GAE를 사용

$$\begin{matrix}
R_t+\gamma V(s_{t+1})-V(s_t) \text{  : 1-step TD error} \\
R_t+\gamma R_{t+1}+\gamma^2V(s_{t+2})-V(s_t) \text{  : 2-step TD error} \\
\vdots \\
\end{matrix}$$

1개만 이용하는게 아니라 exponential moving average 하자.

간단하게 나타내기 위해 $A_t^{(1)}=\delta_t$로 치환하자 

$$A_t^{(2)}=R_t+\gamma R_{t+1}+\gamma^2V(s_{t+2})-V(s_t)=(R_t+\gamma V(s_{t+1})-V(s_t))+\gamma(R_{t+1}+\gamma V(s_{t+2})-V(s_{t+1}))=\delta_t+\gamma\delta_{t+1}$$

$$A_t^{(n)}=\sum_{k=t}^{t+n-1}\gamma^{k-t}\delta_k$$ 로 표현할 수 있다.

$$TD(\lambda)\rightarrow\sum^\infty_{n=1}(1-\lambda)\lambda^{n-1}A_t^{(n)}$$를 이와 같이 표현하면

$$\begin{matrix}
TD(\lambda)&=(1-\lambda)(\delta_t+\lambda(\delta_t+\gamma\delta_{t+1})+\lambda^2(\delta_t+\gamma\delta_{t+1}+\gamma^2\delta_{t+2})+\cdots) \\
&=(1-\lambda)(\delta_t(1+\lambda+\lambda^2+\cdots)+\gamma\delta_{t+1}(\lambda+\lambda^2+\cdots)+\cdots) \\
&=(1-\lambda)(\delta_t*\frac{1}{1-\lambda}+\gamma\delta_{t+1}*\frac{\lambda}{1-\lambda}+\cdots) \\ 
&=\sum_{k=t}^{\infty}(\gamma\lambda)^{k-t}\delta_k
\end{matrix}$$로 표현될 수 있다.

하지만 몬테카를로가 아니라 n-step TD이기 때문에 $\infty$를 계산할 수 없다. 따라서 근사를 사용

$$\begin{matrix}
A_i^{GAE}=\sum_{k=i}^{\infty}(\gamma\lambda)^{k-i}\delta_k \\
\hat{A}_i^{GAE}=\sum_{k=i}^{t}(\gamma\lambda)^{k-i}\delta_k
\end{matrix}$$

<img src='https://i.imgur.com/rd5tda1.png' width='800'> 

<img src='../img/ppo.png' width='800'> 

## PPO

### Libraries

For this example the following libraries are used:

1. numpy for n-dimensional arrays
2. tensorflow and keras for building the deep RL PPO agent
3. gym for getting everything we need about the environment
4. scipy.signal for calculating the discounted cumulative sums of vectors

In [37]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical
import scipy.signal
from mpi4py import MPI
import time
import os
import sys
import subprocess
from gym.spaces import Box, Discrete

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Functions and class

- Discount Factor 적용

In [4]:
def discounted_cumulative_sums(x, discount):
    """
    magic from rllab for computing discounted cumulative sums of vectors.
    input: 
        vector x, 
        [x0, 
         x1, 
         x2]
    output:
        [x0 + discount * x1 + discount^2 * x2,  
         x1 + discount * x2,
         x2]
    """
    return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]

```pyhon
scipy.signal.lfilter(b, a, x, axis=- 1, zi=None)
```

$$a[0]*y[n] = b[0]*x[n] + b[1]*x[n-1] + ... + b[M]*x[n-M]
                      - a[1]*y[n-1] - ... - a[N]*y[n-N]$$

- step size X dim shape 생성 function

In [54]:
def combined_shape(length, shape=None):
    if shape is None:
        return (length,)
    return (length, shape) if np.isscalar(shape) else (length, *shape)

- Buffer 생성, 저장, 추출 기능을 제공하는 class

In [57]:
class PPOBuffer:
    # Buffer for storing trajectories
    def __init__(self, obs_dim, act_dim, size, gamma=0.99, lam=0.95):
        # Buffer initialization
        self.observation_buffer = np.zeros(combined_shape(size, obs_dim), dtype=np.float32)
        self.action_buffer = np.zeros(combined_shape(size, act_dim), dtype=np.float32)
        self.advantage_buffer = np.zeros(size, dtype=np.float32)
        self.reward_buffer = np.zeros(size, dtype=np.float32)
        self.return_buffer = np.zeros(size, dtype=np.float32)
        self.value_buffer = np.zeros(size, dtype=np.float32)
        self.logprobability_buffer = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.pointer, self.trajectory_start_index, self.max_size = 0, 0, size

    def store(self, observation, action, reward, value, logprobability):
        # Append one step of agent-environment interaction
        self.observation_buffer[self.pointer] = observation
        self.action_buffer[self.pointer] = action
        self.reward_buffer[self.pointer] = reward
        self.value_buffer[self.pointer] = value
        self.logprobability_buffer[self.pointer] = logprobability
        self.pointer += 1

    def finish_trajectory(self, last_value=0):
        # Finish the trajectory by computing advantage estimates and rewards-to-go
        path_slice = slice(self.trajectory_start_index, self.pointer)
        rewards = np.append(self.reward_buffer[path_slice], last_value)
        values = np.append(self.value_buffer[path_slice], last_value)

        deltas = rewards[:-1] + self.gamma * values[1:] - values[:-1]

        self.advantage_buffer[path_slice] = discounted_cumulative_sums(
            deltas, self.gamma * self.lam
        )
        self.return_buffer[path_slice] = discounted_cumulative_sums(
            rewards, self.gamma
        )[:-1]

        self.trajectory_start_index = self.pointer
    
    def get(self):
        # Get all data of the buffer and normalize the advantages
        assert self.pointer == self.max_size
        self.pointer, self.trajectory_start_index = 0, 0
        advantage_mean, advantage_std = (
            np.mean(self.advantage_buffer),
            np.std(self.advantage_buffer),
        )
        self.advantage_buffer = (self.advantage_buffer - advantage_mean) / advantage_std
        data = dict(obs=self.observation_buffer, 
                    act=self.action_buffer, 
                    ret=self.return_buffer,
                    adv=self.advantage_buffer, 
                    logp=self.logprobability_buffer)
        return {k: torch.as_tensor(v, dtype=torch.float32) for k,v in data.items()}


- 모델 네트워크 선언

In [6]:
def mlp(sizes, activation, output_activation=nn.Identity):
    layers = []
    for j in range(len(sizes)-1):
        act = activation if j < len(sizes)-2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j+1]), act()]
    return nn.Sequential(*layers)


class Actor(nn.Module):

    def _distribution(self, obs):
        raise NotImplementedError

    def _log_prob_from_distribution(self, pi, act):
        raise NotImplementedError

    def forward(self, obs, act=None):
        # Produce action distributions for given observations, and 
        # optionally compute the log likelihood of given actions under
        # those distributions.
        pi = self._distribution(obs)
        logp_a = None
        if act is not None:
            logp_a = self._log_prob_from_distribution(pi, act)
        return pi, logp_a


class MLPCategoricalActor(Actor):
    
    def __init__(self, obs_dim, act_dim, hidden_sizes, activation):
        super().__init__()
        self.logits_net = mlp([obs_dim] + list(hidden_sizes) + [act_dim], activation)

    def _distribution(self, obs):
        logits = self.logits_net(obs)
        return Categorical(logits=logits)

    def _log_prob_from_distribution(self, pi, act):
        return pi.log_prob(act)


class MLPGaussianActor(Actor):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation):
        super().__init__()
        log_std = -0.5 * np.ones(act_dim, dtype=np.float32)
        self.log_std = torch.nn.Parameter(torch.as_tensor(log_std))
        self.mu_net = mlp([obs_dim] + list(hidden_sizes) + [act_dim], activation)

    def _distribution(self, obs):
        mu = self.mu_net(obs)
        std = torch.exp(self.log_std)
        return Normal(mu, std)

    def _log_prob_from_distribution(self, pi, act):
        return pi.log_prob(act).sum(axis=-1)    # Last axis sum needed for Torch Normal distribution
    
    
class MLPCritic(nn.Module):

    def __init__(self, obs_dim, hidden_sizes, activation):
        super().__init__()
        self.v_net = mlp([obs_dim] + list(hidden_sizes) + [1], activation)

    def forward(self, obs):
        return torch.squeeze(self.v_net(obs), -1) # Critical to ensure v has right shape.

In [7]:
class MLPActorCritic(nn.Module):


    def __init__(self, observation_space, action_space, 
                 hidden_sizes=(64,64), activation=nn.Tanh):
        super().__init__()

        obs_dim = observation_space.shape[0]

        # policy builder depends on action space
        if isinstance(action_space, Box):
            self.pi = MLPGaussianActor(obs_dim, action_space.shape[0], hidden_sizes, activation)
        elif isinstance(action_space, Discrete):
            self.pi = MLPCategoricalActor(obs_dim, action_space.n, hidden_sizes, activation)

        # build value function
        self.v  = MLPCritic(obs_dim, hidden_sizes, activation)

    def step(self, obs):
        with torch.no_grad():
            pi = self.pi._distribution(obs)
            a = pi.sample()
            logp_a = self.pi._log_prob_from_distribution(pi, a)
            v = self.v(obs)
        return a.numpy(), v.numpy(), logp_a.numpy()

    def act(self, obs):
        return self.step(obs)[0]

In [89]:
from torchviz import make_dot
from torch.autograd import Variable

ac = MLPActorCritic(gym.make("CartPole-v0").observation_space, 
                    gym.make("CartPole-v0").action_space, 
                    hidden_sizes=hidden_sizes)
act=0
env = gym.make("CartPole-v0")
env.reset()
obs, _, _, _ = env.step(act)
# make_dot(ac.pi(torch.as_tensor(obs, dtype=torch.float32), torch.as_tensor(act)), 
#          params=dict(ac.pi.named_parameters())).render("../img/ppo_pi", format="png")
make_dot(ac.v(torch.as_tensor(obs, dtype=torch.float32)), 
         params=dict(ac.v.named_parameters())).render("../img/ppo_v", format="png")

'../img/ppo_v.png'

<img src='../img/ppo_v.png' width='400'> 

- 분산 및 병렬처리를 위한 MPI 사용 function 선언
[mpi4py example](https://fortran.readthedocs.io/ko/latest/mpi4py_ex/)

In [63]:
def num_procs():
    """Count active MPI processes."""
    return MPI.COMM_WORLD.Get_size()

def setup_pytorch_for_mpi():
    """
    Avoid slowdowns caused by each separate process's PyTorch using
    more than its fair share of CPU resources.
    """
    #print('Proc %d: Reporting original number of Torch threads as %d.'%(proc_id(), torch.get_num_threads()), flush=True)
    if torch.get_num_threads()==1:
        return
    fair_num_threads = max(int(torch.get_num_threads() / num_procs()), 1)
    torch.set_num_threads(fair_num_threads)
    #print('Proc %d: Reporting new number of Torch threads as %d.'%(proc_id(), torch.get_num_threads()), flush=True)
    
def sync_params(module):
    """ Sync all parameters of module across all MPI processes. """
    if num_procs()==1:
        return
    for p in module.parameters():
        p_numpy = p.data.numpy()
        broadcast(p_numpy)
        
def mpi_avg_grads(module):
    """ Average contents of gradient buffers across MPI processes. """
    if num_procs()==1:
        return
    for p in module.parameters():
        p_grad_numpy = p.grad.numpy()   # numpy view of tensor data
        avg_p_grad = mpi_avg(p.grad)
        p_grad_numpy[:] = avg_p_grad[:]
        
def mpi_fork(n, bind_to_core=False):
    """
    Re-launches the current script with workers linked by MPI.
    Also, terminates the original process that launched it.
    Taken almost without modification from the Baselines function of the
    `same name`_.
    .. _`same name`: https://github.com/openai/baselines/blob/master/baselines/common/mpi_fork.py
    Args:
        n (int): Number of process to split into.
        bind_to_core (bool): Bind each MPI process to a core.
    """
    if n<=1: 
        return
    if os.getenv("IN_MPI") is None:
        env = os.environ.copy()
        env.update(
            MKL_NUM_THREADS="1",
            OMP_NUM_THREADS="1",
            IN_MPI="1"
        )
        args = ["mpirun", "-np", str(n)]
        if bind_to_core:
            args += ["-bind-to", "core"]
        args += [sys.executable] + sys.argv
        subprocess.check_call(args, env=env)
        sys.exit()
        
def allreduce(*args, **kwargs):
    return MPI.COMM_WORLD.Allreduce(*args, **kwargs)
    
def mpi_op(x, op):
    x, scalar = ([x], True) if np.isscalar(x) else (x, False)
    x = np.asarray(x, dtype=np.float32)
    buff = np.zeros_like(x, dtype=np.float32)
    allreduce(x, buff, op=op)
    return buff[0] if scalar else buff
        
def mpi_sum(x):
    return mpi_op(x, MPI.SUM)        
        
def mpi_avg(x):
    """Average a scalar or vector over MPI processes."""
    return mpi_sum(x) / num_procs()

- PPO training

In [67]:
def ppo(env_fn, actor_critic=MLPActorCritic, ac_kwargs=dict(), seed=702, 
        steps_per_epoch=4000, epochs=50, gamma=0.99, clip_ratio=0.2, pi_lr=3e-4,
        vf_lr=1e-3, train_pi_iters=80, train_v_iters=80, lam=0.97, max_ep_len=1000,
        target_kl=0.01):
    # Special function to avoid certain slowdowns from PyTorch + MPI combo.
    setup_pytorch_for_mpi()

    # Random seed
    seed = 702
    torch.manual_seed(seed)
    np.random.seed(seed)

    # Instantiate environment
    env = env_fn()
    obs_dim = env.observation_space.shape[0]
    act_dim = env.action_space.shape

    # Create actor-critic module
    ac = actor_critic(env.observation_space, env.action_space, **ac_kwargs)

    # Sync params across processes
    sync_params(ac)

    # Count variables
    var_counts = tuple(sum([np.prod(p.shape) for p in module.parameters()]) for module in [ac.pi, ac.v])

    # Set up experience buffer
    local_steps_per_epoch = int(steps_per_epoch / num_procs())
    buf = PPOBuffer(obs_dim, act_dim, local_steps_per_epoch, gamma, lam)

    # Set up function for computing PPO policy loss
    def compute_loss_pi(data):
        obs, act, adv, logp_old = data['obs'], data['act'], data['adv'], data['logp']

        # Policy loss
        pi, logp = ac.pi(obs, act)
        ratio = torch.exp(logp - logp_old)
        clip_adv = torch.clamp(ratio, 1-clip_ratio, 1+clip_ratio) * adv
        loss_pi = -(torch.min(ratio * adv, clip_adv)).mean()

        # Useful extra info
        approx_kl = (logp_old - logp).mean().item()
        ent = pi.entropy().mean().item()
        clipped = ratio.gt(1+clip_ratio) | ratio.lt(1-clip_ratio)
        clipfrac = torch.as_tensor(clipped, dtype=torch.float32).mean().item()
        pi_info = dict(kl=approx_kl, ent=ent, cf=clipfrac)

        return loss_pi, pi_info

    # Set up function for computing value loss
    def compute_loss_v(data):
        obs, ret = data['obs'], data['ret']
        return ((ac.v(obs) - ret)**2).mean()

    # Set up optimizers for policy and value function
    pi_optimizer = Adam(ac.pi.parameters(), lr=pi_lr)
    vf_optimizer = Adam(ac.v.parameters(), lr=vf_lr)

    
    def update():
        data = buf.get()

        pi_l_old, pi_info_old = compute_loss_pi(data)
        pi_l_old = pi_l_old.item()
        v_l_old = compute_loss_v(data).item()

        # Train policy with multiple steps of gradient descent
        for i in range(train_pi_iters):
            pi_optimizer.zero_grad()
            loss_pi, pi_info = compute_loss_pi(data)
            kl = mpi_avg(pi_info['kl'])
            if kl > 1.5 * target_kl:
                break
            loss_pi.backward()
            mpi_avg_grads(ac.pi)    # average grads across MPI processes
            pi_optimizer.step()


        # Value function learning
        for i in range(train_v_iters):
            vf_optimizer.zero_grad()
            loss_v = compute_loss_v(data)
            loss_v.backward()
            mpi_avg_grads(ac.v)    # average grads across MPI processes
            vf_optimizer.step()


    # Prepare for interaction with environment
    start_time = time.time()
    o, ep_ret, ep_len = env.reset(), 0, 0

    # Main loop: collect experience in env and update/log each epoch
    for epoch in range(epochs):
        for t in range(local_steps_per_epoch):
            a, v, logp = ac.step(torch.as_tensor(o, dtype=torch.float32))

            next_o, r, d, _ = env.step(a)
            ep_ret += r
            ep_len += 1

            # save and log
            buf.store(o, a, r, v, logp)
            
            # Update obs (critical!)
            o = next_o

            timeout = ep_len == max_ep_len
            terminal = d or timeout
            epoch_ended = t==local_steps_per_epoch-1

            if terminal or epoch_ended:
                if epoch_ended and not(terminal):
                    print(f'Epoch: {epoch} Warning: trajectory cut off by epoch at {ep_len} steps.', flush=True)
                # if trajectory didn't reach terminal state, bootstrap value target
                if timeout or epoch_ended:
                    _, v, _ = ac.step(torch.as_tensor(o, dtype=torch.float32))
                else:
                    v = 0
                buf.finish_trajectory(v)
                o, ep_ret, ep_len = env.reset(), 0, 0


        # Perform PPO update!
        update()

### Hyperparameters

In [65]:
# Hyperparameters of the PPO algorithm
steps_per_epoch = 4000
epochs = 100
gamma = 0.99
clip_ratio = 0.2
policy_learning_rate = 3e-4
value_function_learning_rate = 1e-3
train_policy_iterations = 80
train_value_iterations = 80
lam = 0.97
target_kl = 0.01
hidden_sizes = (64, 64)
num_cpu = 1
# True if you want to render the environment
render = False

### Training

In [68]:
# Initialize the environment and get the dimensionality of the
# observation space and the number of possible actions
env = gym.make("CartPole-v0")
mpi_fork(num_cpu)

ppo(lambda : env, actor_critic=MLPActorCritic,
        ac_kwargs=dict(hidden_sizes=hidden_sizes), gamma=gamma, 
        seed=702, steps_per_epoch=steps_per_epoch, epochs=epochs)

Epoch: 0 Warning: trajectory cut off by epoch at 20 steps.
Epoch: 1 Warning: trajectory cut off by epoch at 9 steps.
Epoch: 2 Warning: trajectory cut off by epoch at 38 steps.
Epoch: 3 Warning: trajectory cut off by epoch at 7 steps.
Epoch: 4 Warning: trajectory cut off by epoch at 32 steps.
Epoch: 5 Warning: trajectory cut off by epoch at 5 steps.
Epoch: 6 Warning: trajectory cut off by epoch at 56 steps.
Epoch: 7 Warning: trajectory cut off by epoch at 119 steps.
Epoch: 8 Warning: trajectory cut off by epoch at 12 steps.
Epoch: 9 Warning: trajectory cut off by epoch at 141 steps.
Epoch: 10 Warning: trajectory cut off by epoch at 53 steps.
Epoch: 11 Warning: trajectory cut off by epoch at 19 steps.
Epoch: 12 Warning: trajectory cut off by epoch at 132 steps.
Epoch: 13 Warning: trajectory cut off by epoch at 69 steps.
Epoch: 14 Warning: trajectory cut off by epoch at 198 steps.
Epoch: 15 Warning: trajectory cut off by epoch at 48 steps.
Epoch: 19 Warning: trajectory cut off by epoch at

### visualization

<img src='https://i.imgur.com/tKhTEaF.gif' width='600'> 

---